# Leeds Africa Hackathon
This is a template notebook for the Leeds Africa Hackathon. It also includes some useful Python code to read CMIP6 data and CP4 data and perform percentile stats (stats_utils.py) as used for Fitzpatrick et al. Climatic Change 2020.

**CMIP6:** 

Historical runs (1850-2014) are in /badc/cmip6/data/CMIP6/CMIP/*institute*/*model*/historical/*variant*/*table_id*/*variable*/*grid*/*version*/

Scenario runs are in /badc/cmip6/data/CMIP6/ScenarioMIP/*institute*/*model*/*scenario_name*/*variant*/*table_id*/*variable*/*grid*/*version*/
*scenario_name* is likely to be one of ssp119, ssp126, ssp245, ssp370 or ssp585

*variant* takes the form r0i0p0f0, e.g. r1i1p1f2, where the numbers are the indexes for:  
**r**ealization, **i**nitialization, **p**hysics, **f**orcing

*table_id* generally indicates the frequency of the data, e.g. day, 3hr, Amon

*grid* is the model grid being used, e.g. gn, where  
   * gm = global mean data  
   * gn = data reported on a model's native grid  
   * gr1 = regridded data reported on a grid other than the native grid and other than the preferred target grid  
It is likely the *grid* will be the native grid, i.e. gn

*version* normally in the form vYYYYMMDD or latest, e.g. v20200203


The following institutions have data in both historical and ScenarioMIPs:
AS-RCEC, AWI, BCC, CAMS, CAS, CCCR-IITM, CCCma, CMCC, CNRM-CERFACS, CSIRO, CSIRO-ARCCSS, E3SM-Project, EC-Earth-Consortium, FIO-QLNM, HAMMOZ-Consortium, INM, IPSL, KIOST, MIROC, MOHC, MPI-M, MRI, NASA-GISS, NCAR, NCC, NIMS-KMA, NOAA-GFDL, NUIST, THU, UA

Any CMIP6 data that is pre-processed should be stored in the group workspace /gws/pw/j05/cop26_hackathons/leeds/. For data that is useful to all teams this should be organised in a similar way to the CMIP6 data so it is clear where it came form, i.e. for future scenarios this is /gws/pw/j05/cop26_hackathons/leeds/<*institute*>/<*model*>/<*scenario_name*>/<*variant*>/<*version*>/<*variable*>/.
For historical data this is
/gws/pw/j05/cop26_hackathons/leeds/<*institute*>/<*model*>/historical/<*variant*>/<*version*>/<*variable*>/.
For data that is specific to the team, please store it under /gws/pw/j05/cop26_hackathons/leeds/*TeamName*/.

We have pre-processed some HadGEM3 data to test our provided stats code and stored it in the group workspace. For example the 50th percentile of future tasmax is stored in tasmax_50th_percentile_2050-2079_*{m}*_*{region_str}*  (2050-2079 was used for future and 1985-2014 for current), *m* is is the month and *region_str* is the region for which we extracted data covering Africa, i.e. 20W55E_30S30N.

**CP4:**

explicit 4km data are in /badc/impala/data/explicit-4km/<*present_or_future*>/<*1hr_or_mon*>/<*variable*>/

parameterised 25km data are in /badc/impala/data/param-25km/<*present_or_future*>/<*1hr_or_mon*>/<*variable*>/

It takes a long time to process hourly 4km data. Therefore we have pre-processed CP4 hourly tas data to create daily tasmax, tasmin and tas mean data regridded to 25km. This has been done for West Africa 20W20E_00N25N and East Africa 20E55E_15SN15N. These are stored in /gws/pw/j05/cop26_hackathons/leeds/CP4/explicit-4km/<*present_or_future*>/ under tasmax, tasmin and tas. We have also created daily mean rainrates under /gws/pw/j05/cop26_hackathons/leeds/CP4/explicit-4km/<*present_or_future*>/pr/. There is one file per month and year. We have done the same for param-25km.



In [ ]:
!ls /badc/cmip6/data/CMIP6/ScenarioMIP/*/*/ssp585/*/day/tas/gn/

In [ ]:
!ls /badc/cmip6/data/CMIP6/CMIP/MOHC/HadGEM3-GC31-LL/historical/r1i1p1f3/day/tasmax/gn/v20190624

In [ ]:
!ls /gws/pw/j05/cop26_hackathons/leeds/CMIP6/MOHC/HadGEM3-GC31-LL/ssp585/r1i1p1f3/v20200114

In [ ]:
!ls /badc/impala/data/explicit-4km/future/1hr/precipitation/

In [ ]:
!ls /gws/pw/j05/cop26_hackathons/leeds/CP4/explicit-4km/future/tasmax

In [ ]:
!ls /gws/pw/j05/cop26_hackathons/leeds/CMIP6/MOHC/HadGEM3-GC31-LL/ssp585/r1i1p1f3/v20200114

In [ ]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading daily tas for the Met office HadGEM3-GC31-LL model for the SSP585 scenario
# from the CMIP6 archive
#-----------------------------------------------------------------------------------------------------
from read_cmip6 import *
import datetime
tas=read_cmip6('ScenarioMIP','MOHC','HadGEM3-GC31-LL','ssp585','r1i1p1f3','day','tas','gn','v20200114','20150101-20491230')
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [ ]:
#-----------------------------------------------------------------------------------------------------
# Here is an example reading present hourly tas for the 25km parameterised CP4 archive
# this only reads a few months as it takes a long time to read all 10 years
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=1997
start_month=3
end_year=1997
end_month=6
tas=read_cp4(False,'1hr','tas', start_year, start_month, end_year, end_month)
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [ ]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading all 10 future years of monthly precip for the 4km explicit CP4 archive
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=2095
start_month=3
end_year=2105
end_month=2
pr=read_cp4(True, 'mon','pr', start_year, start_month, end_year, end_month)
print(pr)
time_coord = pr.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [ ]:
#-----------------------------------------------------------------------------------------------------
# This plots 50 percentiles for Africa for a HadGEM3 model for current and ssp585 future scenario,
# the difference, and where the 50th percentile of ssp585 sits within the current variability
#-----------------------------------------------------------------------------------------------------
from plot_cmip6_tasmax_50th_percentile import *

institute='MOHC'
model='HadGEM3-GC31-LL'
fc_experiment='ssp585'
cc_experiment='historical'
variant='r1i1p1f3'
fc_version='v20200114'
cc_version='v20190624'
lon_range=[-20,55]
lat_range=[-30,30]
fc_year_range=[2050,2079]
cc_year_range=[1985,2014]
levels_diff = np.arange(0.5, 6, 0.25)
plot_cmip6_tasmax_50th_percentile(institute, model, fc_experiment, cc_experiment, variant, fc_version, cc_version, lat_range, lon_range, fc_year_range, cc_year_range,5, levels_diff)

In [ ]:
#-----------------------------------------------------------------------------------------------------
# This plots 50 percentiles for Africa for CP4 explicit-4km for current and future scenario,
# the difference, and where the 50th percentile of ssp585 sits within the current variability
#-----------------------------------------------------------------------------------------------------
from plot_cp4_tasmax_50th_percentile import *

lon_range=[-20,20]
lat_range=[0,25]
levels_diff = np.arange(3, 8, 0.25)
plot_cp4_tasmax_50th_percentile(True, lat_range, lon_range, 5, levels_diff)